# Theoretical Analysis
> Theoretical Analysis (level 0) for all CNN topologies and hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [Rooflines,MNIST,ImageNet,CIFAR-10]
- image: images/roofline.png

In [1]:
#hide
import numpy as np
import pandas as pd

pd.options.display.max_rows = 10000 # this will set limit of columns to 500
pd.options.display.float_format = '{:20,.0f}'.format
import altair as alt
W = 600
H = 480

In [2]:
#hide
#notes
#rn50_df = df[df.apply(lambda row: row['net_prun_quant'].split('_')[0] == 'MobileNetv1|ResNet50', axis=1)]
#rn50_df['net_prun_quant'] = rn50_df['net_prun_quant'].str.split('_').str[0] #comment this when I want to hide the topology
#rn50_df['hardw'] = rn50_df['hardw'].str.replace("_INT8|_FP16|_FP32", "")

# Introduction

This page presents a Theoretical Analysis of both hardware platforms as well as CNN topologies.
In order to get a general overview of all CNNs and Hardware Platforms included in our experiments, we present the following 3 tables. 

# Tables

### CNNs and Their Accuracy Over All Pruning and Quantization Variants

Table below provides a complete overview of all CNNs that were included in the experimentation and their corresponding accuracy over all Pruning and Quantization Variants.

In [3]:
#hide_input
%run scripts/script_tables.py  #run the heatmaps script
tableOverviewExperiments(['data/cnn_topologies_accuracy.csv'])

,INT2,INT4,INT8,FP16,FP32
,top1 (top5) [%],top1 (top5) [%],top1 (top5) [%],top1 (top5) [%],top1 (top5) [%]
GoogLeNetv1,nm,nm,69.24 (88.45),66.93 (87.83),66.96 (87.84)
MobileNetv1,nm,nm,69.57 (87.71),nm,nm
EfficientNet small,nm,nm,77,nm,nm
EfficientNet medium,nm,nm,78.6,nm,nm
EfficientNet large,nm,nm,80.2,nm,nm
ResNet50 100%,nm,nm,73.29 (91.26),75.14 (92.12),75.15 (92.11)
ResNet50 80%,nm,nm,73.30 (91.40),nm,nm
ResNet50 50%,nm,nm,69.49 (91.00),nm,nm
ResNet50 30%,nm,nm,68.83 ( 90.16),nm,nm


In [4]:
#hide
# GET THEORETICAL TABLES
#get the table above
df = pd.read_csv('data/cnn_topologies_accuracy.csv')
df1 = pd.DataFrame()
columns = (df.loc[:, df.columns!=' ']).columns #select all columns except first
for column in columns:
    df_=pd.melt(df, id_vars=[' '], value_vars=column) #melt df1 to have only 2 columns
    df1=pd.concat([df1,df_])
df1.columns= ['net_prun_quant','quant','top1'] #setting new column names
df1 = df1[df1['top1'] !='top1 (top5) [%]']  #removing trash
df1 = df1.reset_index()
df1.net_prun_quant = df1.net_prun_quant + ' ' + df1.quant  #changing column names
df1 = df1.drop(columns=['index','quant'])
df1 = df1[df1.top1!='nm'] # take all 'nm' out
df1['top1'] = df1['top1'].str.split(' ').str[0] #take top5 acc out
df1['net_prun_quant'] = df1['net_prun_quant'].str.replace(' ','_') #put underscore instead of space
#df1

In [5]:
#hide
# GET EXPERIMENTAL TABLES
df2 = pd.read_csv("data/cleaned_csv/performance_prediction_cifar10.csv")
df3 = pd.read_csv("data/cleaned_csv/performance_prediction_imagenet.csv")
df4 = pd.read_csv("data/cleaned_csv/performance_prediction_mnist.csv")
df2= pd.concat([df2, df3, df4])
#df2.head(1000)

In [6]:
#hide
# PROCESS THEM
df2['x'] = df2['x'].str.replace(' ','_')
df2['x']= df2['x'].str.replace('-','_')
df2 = df2[df2['values'].notna()]
df2.columns=['y','net_prun_quant','values']
#df2.head(20)

In [7]:
#hide
# MERGE THEORETICAL WITH EXPERIMENTAL TABLES
df=pd.merge(df1, df2, on='net_prun_quant', how='outer')
df.columns = ['net_prun_quant', 'top1', 'hardw', 'fps']
df = df[df['fps'].notna()]
df = df[df['top1'].notna()]
df['hardw'] = df['hardw'].str.replace('-','_') #put underscore instead of space

#df.head(1000)

In [8]:
#hide
#-----------------PLOTS!!!------------------

In [9]:
#hide
# CIFAR-10
cnv_df = df[df.apply(lambda row: row['net_prun_quant'].split('_')[0] == 'CNV', axis=1)]
cnv_df['net_prun_quant'] = cnv_df['net_prun_quant'].str.split('_').str[0] #comment this when I want to hide the topology
cnv_df.net_prun_quant= cnv_df.net_prun_quant +'_'+ cnv_df.hardw 
cnv_df=cnv_df.drop(columns=['hardw'])
#cnv_df.head(200)
#hide
%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph_points(df= cnv_df, 
             groupcol= 'net_prun_quant', 
             xcol= 'fps', 
             ycol= 'top1', 
             W= 0.1 *W, 
             H= 0.1 *H, 
             title= "CIFAR-10 Cassification Design Space: Accuracy versus Performance")

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


alt.Chart(...)

In [10]:
#hide
# MNIST
mlp_df = df[df.apply(lambda row: row['net_prun_quant'].split('_')[0] == 'MLP', axis=1)]
mlp_df['net_prun_quant'] = mlp_df['net_prun_quant'].str.split('_').str[0] #comment this when I want to hide the topology
mlp_df.net_prun_quant= mlp_df.net_prun_quant +'_'+ mlp_df.hardw
mlp_df=mlp_df.drop(columns=['hardw'])
#mlp_df.head(200)

#hide
%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph_points(df= mlp_df, 
             groupcol= 'net_prun_quant', 
             xcol= 'fps', 
             ycol= 'top1', 
             W= 0.1* W, 
             H= 0.1* H, 
             title= "MNIST Cassification Design Space: Accuracy versus Performance")

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


alt.Chart(...)

In [11]:
#hide
def get_point_chart(df: pd.DataFrame, 
                    color_groupcol: str, 
                    shape_groupcol: str, 
                    size_groupcol:str, 
                    scheme: str,  
                    xcol: str, 
                    ycol: str, 
                    legend_title_groupcol: str)->alt.vegalite.v4.api.Chart: 
    return alt.Chart(df).interactive().mark_point(size=50, opacity=1, filled =True).properties(
            width= W,
            height= H,
            title='  '
        ).encode(
            x= xcol,
            y=alt.Y(ycol + ":Q", scale=alt.Scale(zero=False)),
            color=alt.Color(color_groupcol,scale=alt.Scale(scheme=scheme), legend=alt.Legend(columns=2, title = legend_title_groupcol)),
            #tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", ycol, xcol],
            shape=alt.Shape(shape_groupcol),
            tooltip=[color_groupcol, shape_groupcol, xcol, ycol],
            size = alt.Size(size_groupcol, legend=None)
        )

In [12]:
#hide
def get_point_chart_side_by_side(df: pd.DataFrame, color_groupcol: str, shape_groupcol: str, size_groupcol:str, scheme: str,  xcol: str, ycol: str, column: str)->alt.vegalite.v4.api.Chart: 
    return alt.Chart(df).interactive().mark_point(size=100, opacity=1, filled =True).encode(
            x= xcol,
            y=alt.Y(ycol + ":Q", scale=alt.Scale(zero=False)),
            color=alt.Color(color_groupcol), #,scale=alt.Scale(scheme=scheme)),
            column= column,
            shape=alt.Shape(shape_groupcol),
            tooltip=[color_groupcol, shape_groupcol, xcol, ycol],
        )

In [13]:
#hide
def get_line_chart(df: pd.DataFrame, groupcol: str, xcol: str, ycol:str, color:str) ->alt.vegalite.v4.api.Chart:
    """Creates simple line chart"""
    return alt.Chart(df).interactive().mark_line(point=True).encode(
        x=xcol,
        y=alt.Y(ycol + ":Q", scale=alt.Scale(zero=False)),
        color=alt.value(color),
        tooltip=[groupcol, xcol, ycol],
    )

In [14]:
#hide
def get_pareto_df(df: pd.DataFrame(), groupcol: str, xcol: str, ycol: str) -> pd.DataFrame():
    """Creates a pareto line from the dataframe. This function doesn't correctly correspond x to y datapoints"""
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

In [15]:
#hide
def get_several_paretos_df(list_df: pd.DataFrame, groupcol: str, xcol: str, ycol:str, colors: list)->list:
    df_out_charts = pd.DataFrame(columns=['charts'])
    for i, df in enumerate(list_df):
        pareto_df = get_pareto_df(df= df , groupcol= groupcol, xcol= xcol, ycol= ycol)
        chart = get_line_chart(df= pareto_df, groupcol= 'group', xcol= 'x', ycol= 'y', color = colors[i]) 
        df_out_charts = df_out_charts.append(pd.DataFrame([[chart]], columns=['charts']))
    return df_out_charts

In [16]:
#hide
# IMAGENET
rn50_df = df[df["net_prun_quant"].str.contains("MobileNetv1|ResNet50", na=False)]
rn50_df['network'] = rn50_df['net_prun_quant'].str.split('_').str[0]
rn50_df['net_prun_quant'] = rn50_df['net_prun_quant'].str.replace("_FP16|_FP32|_INT8", "")
rn50_df.net_prun_quant = rn50_df.hardw +'_'+  rn50_df.net_prun_quant 
#print(rn50_df)
rn50_df['net_prun_quant'] = rn50_df['net_prun_quant'].str.replace("_DPU", "-DPU")
rn50_df['net_prun_quant'] = rn50_df['net_prun_quant'].str.replace("%", " ")
rn50_df['net_prun_quant'] = rn50_df['net_prun_quant'].str.replace("Ultra96", "Ultra96-DPU")


rn50_df=rn50_df.drop(columns=['hardw'])
rn50_df['type'] = 'theoretical'
rn50_df['hardware'] = rn50_df['net_prun_quant'].str.split('_').str[0]

exper_df = pd.read_csv('data/cleaned_csv/pareto_data_imagenet.csv')
df_pareto = pd.concat([rn50_df, exper_df])
df_pareto['network'] = df_pareto['network'].str.replace("RN50", "ResNet50")
df_pareto['network'] = df_pareto['network'].str.replace("MNv1", "MobileNetv1")
df_pareto['size'] = df_pareto.apply(lambda r: 20 if r.type == 'theoretical' else (120), axis=1)
df_theo = df_pareto.loc[df_pareto['type'] == 'theoretical']
df_exper = df_pareto.loc[df_pareto['type'] == 'experimental']

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [34]:
#hide_input
df_charts = get_several_paretos_df(list_df = [df_theo, df_exper], groupcol='net_prun_quant' , xcol='fps' , ycol='top1', colors=['#FFA500', '#0066CC'])
chart1 = get_point_chart(df= df_exper, color_groupcol= 'net_prun_quant', shape_groupcol= 'type', size_groupcol='size', scheme='greenblue', xcol= 'fps', ycol= 'top1', legend_title_groupcol="Exper_Hrdw_Quant_Net_Prun" )
chart2 = get_point_chart(df= df_theo, color_groupcol= 'net_prun_quant', shape_groupcol= 'type', size_groupcol='size', scheme='purplered', xcol= 'fps', ycol= 'top1', legend_title_groupcol="Theor_Hrdw_Quant_Net_Prun" )

#df_charts = df_charts.append(pd.DataFrame([[chart2],[chart1]], columns=['charts']))
chart = df_charts.charts.sum(numeric_only = False)
alt.layer(
    chart1,
    chart2,
    chart
).resolve_scale(color='independent').properties(title='Overlapped pareto plots Theoretical + Experimental')

alt.LayerChart(...)

In [32]:
#hide_input
chart =get_point_chart_side_by_side(df= df_pareto, color_groupcol= 'net_prun_quant', shape_groupcol= 'network', size_groupcol='size', scheme='purplered', xcol= 'fps', ycol= 'top1', column = 'type')
#chart =get_point_chart_side_by_side(df= df_pareto, color_groupcol= 'network', shape_groupcol= 'net_prun_quant', size_groupcol='size', scheme='purplered', xcol= 'fps', ycol= 'top1', column = 'type')


#df_pareto_charts = get_several_paretos_df(list_df = [df_theo, df_exper], groupcol='net_prun_quant' , xcol='fps' , ycol='top1', colors=['#FFA500', '#0066CC'])
#pareto_charts = df_pareto_charts.charts.sum(numeric_only = False)

chart.properties(title='Overlapped pareto plots Experimental + Theoretical')   # + pareto_charts

alt.Chart(...)

In [19]:
#hide
import altair as alt
from vega_datasets import data

#data_ = pd.read_json(data)

base_wheat = alt.Chart(data.wheat.url).transform_calculate(
    year_end="+datum.year + 5")


bars = base_wheat.mark_bar(**{"fill": "#aaa", "stroke": "#999"}).encode(
    x=alt.X("year:Q", axis=alt.Axis(format='d', tickCount=5)),
    y=alt.Y("wheat:Q", axis=alt.Axis(zindex=1)),
    x2=alt.X2("year_end")
)

area = base_wheat.mark_area(**{"color": "#a4cedb", "opacity": 0.7}).encode(
    x=alt.X("year:Q"),
    y=alt.Y("wages:Q")
)

area_line_1 = area.mark_line(**{"color": "#000", "opacity": 0.7})



(bars + area + area_line_1  ).properties(
    width=900, height=400
).configure_axis(
    title=None, gridColor="white", gridOpacity=0.25, domain=False
).configure_view(
    stroke="transparent"
)

alt.LayerChart(...)

In [20]:
#hide
import altair as alt
from vega_datasets import data


bars = alt.Chart(df_pareto).mark_bar(**{"fill": "#aaa", "stroke": "#999"}).encode(
    x=alt.X("fps:Q", axis=alt.Axis(format='d', tickCount=5)),
    y=alt.Y("top1:Q", axis=alt.Axis(zindex=1)),
    #x2=alt.X2("type")
)

area = alt.Chart(df_pareto).mark_area(**{"color": "#a4cedb", "opacity": 0.7}).encode(
    x=alt.X("fps:Q"),
    y=alt.Y("type:Q")
)

area_line_1 = area.mark_line(**{"color": "#000", "opacity": 0.7})



(bars + area + area_line_1  ).properties(
    width=900, height=400
).configure_axis(
    title=None, gridColor="white", gridOpacity=0.25, domain=False
).configure_view(
    stroke="transparent"
)

alt.LayerChart(...)

In [21]:
#hide
df_pareto

,net_prun_quant,top1,fps,network,type,hardware,batch/thread/stream,size
41,TPU_fast_INT8_MobileNetv1,69.57,"3,509",MobileNetv1,theoretical,TPU,nan,20
42,TPU_slow_INT8_MobileNetv1,69.57,"1,754",MobileNetv1,theoretical,TPU,nan,20
46,Ultra96-DPU_INT8_ResNet50_100,73.29,124,ResNet50,theoretical,Ultra96-DPU,nan,20
47,ZCU104-DPU_INT8_ResNet50_100,73.29,596,ResNet50,theoretical,ZCU104-DPU,nan,20
48,ZCU102-DPU_INT8_ResNet50_100,73.29,753,ResNet50,theoretical,ZCU102-DPU,nan,20
49,TPU_fast_INT8_ResNet50_100,73.29,518,ResNet50,theoretical,TPU,nan,20
50,TPU_slow_INT8_ResNet50_100,73.29,259,ResNet50,theoretical,TPU,nan,20
51,Ultra96-DPU_INT8_ResNet50_80,73.30,147,ResNet50,theoretical,Ultra96-DPU,nan,20
52,ZCU104-DPU_INT8_ResNet50_80,73.30,703,ResNet50,theoretical,ZCU104-DPU,nan,20
53,ZCU102-DPU_INT8_ResNet50_80,73.30,798,ResNet50,theoretical,ZCU102-DPU,nan,20


In [22]:
#hide
import altair as alt
from vega_datasets import data

source = data.barley()

alt.Chart(df_pareto).mark_bar().encode(
    x='fps',
    y='top1',
    color='net_prun_quant',
    tooltip=['net_prun_quant', 'fps', 'top1','type'],
)


alt.Chart(...)

In [23]:
#hide
source


,yield,variety,year,site
0,27,Manchuria,1931,University Farm
1,49,Manchuria,1931,Waseca
2,27,Manchuria,1931,Morris
3,40,Manchuria,1931,Crookston
4,33,Manchuria,1931,Grand Rapids
5,29,Manchuria,1931,Duluth
6,43,Glabron,1931,University Farm
7,55,Glabron,1931,Waseca
8,29,Glabron,1931,Morris
9,38,Glabron,1931,Crookston


### CNNs and Their Compute and Memory Requirements

Next table shows Compute and Memory Requirements for all CNNs in number of operations ([GOPs]), Model Size ([ME]) and Operational Intensity ([OI]) in operations per byte read or written from memory.

In [24]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/cnn_topologies_compute_memory_requirements.csv'])

,Total OPs,Total Model Size,OI (INT2),OI (INT4),OI (INT8),OI (FP16),OI (FP32)
,GOPs,[ME],[Ops/Byte],[Ops/Byte],[Ops/Byte],[Ops/Byte],[Ops/Byte]
GoogLeNetv1,3.1,6,2093.97,1046.99,523.49,261.75,130.87
MobileNetv1,1.1,4.2,1075.47,537.74,268.87,134.43,67.22
ResNet-50 100%,7.7,25.5,1210.84,605.42,302.71,151.36,75.68
ResNet-50 80%,6.5,23.7,1086.59,543.3,271.65,135.82,67.91
ResNet-50 50%,3.8,15.8,949.85,474.93,237.46,118.73,59.37
ResNet-50 30%,2.5,10.1,970.16,485.08,242.54,121.27,60.64
EfficientNet Edge L,4.7,5.4,3481.48,1740.74,870.37,435.18,217.59
EfficientNet Edge M,7.4,6.9,4289.86,2144.93,1072.46,536.23,268.12
EfficientNet Edge S,19.4,10.6,7313.21,3656.6,1828.3,914.15,457.08


## Hardware Platforms

Table below summarizes all included hardware platforms, each with its corresponding peak performance for diferent datatypes (INTx, FPx), its Memory Bandwidth, Memory capacity as well as Thermal Design Power.

In [25]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/hardware_platforms.csv'])

Hardware Platforms,INT2,INT4,INT8,FP16,FP32,Memory Bandwidth,Memory Capacity,Power
,[TOP/sec],[TOP/sec],[TOP/sec],[TOP/sec],[TOP/sec],[GBps],[GB],[Watt]
Ultra96-DPU,na,na,0.96,na,na,4.26,2,na
ZCU104-DPU,na,na,4.6,na,na,19.2,4,na
ZCU102-DPU,na,na,6.71,na,na,19.2,4,na
ZCU104-FINN,30.7,8.8,na,na,na,19.2,4,na
ZCU104-BISMO,30.7,8.8,na,na,na,19.2,4,na
TX2 - maxn,na,na,na,1.33,0.67,59.7,8,15
TX2 - maxp,na,na,na,1.15,0.57,59.7,8,15
TX2 - maxq,na,na,na,0.87,0.44,59.7,8,15
TPU-fast,na,na,4,na,na,25.6,1,2


## Overview of Theoretical Evaluation

link to: https://rcl-lab.github.io/Qutibench_Web/mnist/imagenet/cifar-10/2020/04/30/Overview_of_experiments.html

# Rooflines for all Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [26]:
#hide_input

#first process the following csv's to get clean ready to plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 'imagenet|mnist|cifar')

alt.LayerChart(...)

# Performance Prediction

The following heatmaps show the theoretical performance for the listed hardware platforms across the various machine learning tasks: MNIST, ImageNet and CIFAR-10. The metric used for the theoretical performance is input/second.

In [27]:
#hide
# First process the unfiltered csv and save it as a pretty csv ready to plotted as a heatmap
#%run scripts/script_load_save_data.py
#have to change clean_csv_performance_predictions to process the right csv the right way
#clean_csv_performance_predictions('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/performance_predictions_imagenet_mnist_cifar.csv')

### MNIST

For MNIST, quantization combined with pruning deliver some of best performance results.

In [28]:
#hide_input
%run scripts/altair_plots.py
#load mnist dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_mnist.csv"), 'red', 'Performance Prediction for MNIST')

alt.LayerChart(...)

### ImageNet

For ImageNet, quantization combined with pruning also deliver some of best performance results.

In [29]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load imagenet dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_imagenet.csv"), 'lightgrey','Performance Prediction for Imagenet')

alt.LayerChart(...)

### CIFAR-10

Finally, for CIFAR-10, quantization combined with pruning deliver some of best performance results

In [30]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load cifar10 dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_cifar10.csv"), 'pink','Performance Prediction for CIFAR-10')

alt.LayerChart(...)